In [1]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import mat73

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from filterpy.kalman import KalmanFilter
from scipy.signal import wiener

In [2]:
wiener.filter()
# define constants and helper functions

# file to process
fname = "indy_20160407_02.mat"

# bin width
bin_width = 16e-3

# trial and error to match value in refh_results.csv
min_neuron_len = 383
    
# define a valid neuron
def valid_neuron(neuron):
    return len(neuron) >= min_neuron_len

# compute snr same as in Makin 2018
def compute_snr(r2):
    return -10*np.log10(1-r2)

In [3]:
# load and preprocess data

# load data from a single file into python
data = mat73.loadmat(fname)

# extract variables from the data dictionary
for key,val in data.items():
    exec(key + '=val')

# merge (flatten) the spikes data structure for ease of manipulation
spks = []
for electrode in spikes:
    for neuron in electrode:
        if valid_neuron(neuron):
            spks.append(neuron)

n_neurons = len(spks)

In [4]:
# downsample kinematic data (4ms -> 16ms) and time vector

pos  = cursor_pos[ 3:-1:4 , : ] # start at sample 3 - makes the vector same size as in refh_results.csv
t_d  = t[ 3:-1:4 ] 

In [5]:
# bin the spike data

bins = np.append( t_d , t_d[-1]+bin_width )
binned_spikes = np.zeros( (n_neurons,len(t_d)) )

for i,neuron in enumerate(spks):
    cnt,_ = np.histogram(neuron,bins)
    binned_spikes[i,:] = cnt

In [6]:
# divide into training and testing data sets

t_train_sec     = 320
n_train_samples = int(t_train_sec / bin_width)

pos_train           = pos[ :n_train_samples , : ]
pos_test            = pos[ n_train_samples: , : ]

binned_spikes_train = binned_spikes[ : , :n_train_samples ]
binned_spikes_test  = binned_spikes[ : , n_train_samples: ]

t_train             = t_d [ :n_train_samples ]
t_test              = t_d [ n_train_samples: ]

In [7]:
# verify sizes are as expected

print(f"binned_spikes_train: {binned_spikes_train.shape}")
print(f"pos_train: {pos_train.shape}")
print("")
print(f"binned_spikes_test: {binned_spikes_test.shape}")
print(f"pos_test: {pos_test.shape}")

binned_spikes_train: (291, 20000)
pos_train: (20000, 2)

binned_spikes_test: (291, 31111)
pos_test: (31111, 2)


In [8]:
spikesTrainData = binned_spikes_train[0].reshape(-1,1)
xpositionTrainData = np.transpose(pos_train)[0] # comparing to the monkey's finger x position
ypositionTrainData = np.transpose(pos_train)[1] # comparing to the monkey's finger y position

spikesTestData = binned_spikes_test[0].reshape(-1,1)
xpositionTestData = np.transpose(pos_test)[0]
ypositionTestData = np.transpose(pos_test)[1]

In [9]:
print(binned_spikes_train[0].reshape(-1,1).shape)
print(binned_spikes_train[0].shape)
#print(binned_spikes_train[1:].reshape(-1,1).shape)
#print(data1[0:320].shape)
#print(data2[0:320].shape)
#print(data2y)


(20000, 1)
(20000,)


### Trying Linear Regression

In [10]:
# Trying out LinearRegression() model

def lregModel(data1, data2, spikeTestData, positionTestData):
    lreg = LinearRegression().fit(data1, data2)
    print(lreg)

    r_sq = lreg.score(data1, data2)
    print('Training data coefficient of determination:', r_sq)

    # Make predictions using the testing set
    predictionData = lreg.predict(spikesTestData)

    # The mean squared error
    print("Testing data mean squared error: %.2f" % mean_squared_error(positionTestData, predictionData))
    # The coefficient of determination: 1 is perfect prediction
    print("Testing data coefficient of determination: %.2f" % r2_score(positionTestData, predictionData))

In [11]:
print("Linear regression of neurons firing to Monkey's finger x position:\n")
lregModel(spikesTrainData, xpositionTrainData, spikesTestData, xpositionTestData)

print("\n\n\nLinear regression of neurons firing to Monkey's finger y position:\n")
lregModel(spikesTrainData, ypositionTrainData, spikesTestData, ypositionTestData)

Linear regression of neurons firing to Monkey's finger x position:

LinearRegression()
Training data coefficient of determination: 0.0039943023709970715
Testing data mean squared error: 2416.11
Testing data coefficient of determination: -0.00



Linear regression of neurons firing to Monkey's finger y position:

LinearRegression()
Training data coefficient of determination: 0.006535524747832477
Testing data mean squared error: 697.73
Testing data coefficient of determination: -0.01


### Trying Kalman Filter

In [12]:
f = KalmanFilter (dim_x=len(spikesTrainData), dim_z=1)

# Assign the initial value for the state (position and velocity). You can do this with a two dimensional array like so:
f.x = np.transpose(spikesTrainData)

#Define the state transition matrix:
f.F = binned_spikes_train[1].reshape(-1,1)

# Define the measurement function:
f.H = xpositionTrainData

#Define the covariance matrix. Here I take advantage of the fact that P already contains np.eye(dim_x), and just multiply by the uncertainty:
f.P *= 1000.

#Now assign the measurement noise. Here the dimension is 1x1, so I can use a scalar
f.R = 5

In [13]:
for i in range(20000):
    z = spikesTestData
    f.predict()
    f.update(z)
    print(f.x)

ValueError: shapes (20000,1) and (20000,20000) not aligned: 1 (dim 1) != 20000 (dim 0)

### Trying Wiener Filter